#Part One

###Initializing SparkSession

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

### Creating our dataframe

In [6]:
sen_df = spark.createDataFrame([
  (0, 'Hi I heard about Spark'),
  (1, 'I wish java could use case classes'),
  (2, 'Logistic,regression,models,are,neat')
], ['id', 'sentence'])

In [7]:
sen_df.show()

+---+--------------------+
 id| sentence|
+---+--------------------+
 0|Hi I heard about ...|
 1|I wish java could...|
 2|Logistic,regressi...|
+---+--------------------+

### Creating Tokenizer object

In [9]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [10]:
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')

### Creating a User Defined Function

In [12]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
count_tokens = udf(lambda words:len(words), IntegerType())

###Creating the tokenized dataframe

In [14]:
tokenized = tokenizer.transform(sen_df)

In [15]:
tokenized.show()

+---+--------------------+--------------------+
 id| sentence| words|
+---+--------------------+--------------------+
 0|Hi I heard about ...|[hi, i, heard, ab...|
 1|I wish java could...|[i, wish, java, c...|
 2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+

In [16]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
 id| sentence| words|tokens|
+---+--------------------+--------------------+------+
 0|Hi I heard about ...|[hi, i, heard, ab...| 5|
 1|I wish java could...|[i, wish, java, c...| 7|
 2|Logistic,regressi...|[logistic,regress...| 1|
+---+--------------------+--------------------+------+

### Splitting not only on white space, but in comma too

In [18]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [19]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
 id| sentence| words|tokens|
+---+--------------------+--------------------+------+
 0|Hi I heard about ...|[hi, i, heard, ab...| 5|
 1|I wish java could...|[i, wish, java, c...| 7|
 2|Logistic,regressi...|[logistic, regres...| 5|
+---+--------------------+--------------------+------+

###Stop words removal

In [21]:
from pyspark.ml.feature import StopWordsRemover

In [22]:
sentence_df = spark.createDataFrame([
  (0, ['I', 'saw', 'the', 'green', 'horse']),
  (1, ['Mary', 'had', 'a', 'little', 'lamb'])
], ['id', 'tokens'])

In [23]:
sentence_df.show()

+---+--------------------+
 id| tokens|
+---+--------------------+
 0|[I, saw, the, gre...|
 1|[Mary, had, a, li...|
+---+--------------------+

In [24]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [25]:
remover.transform(sentence_df).show()

+---+--------------------+--------------------+
 id| tokens| filtered|
+---+--------------------+--------------------+
 0|[I, saw, the, gre...| [saw, green, horse]|
 1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+

###n-gram (sequence of n consecutive words)

In [27]:
from pyspark.ml.feature import NGram
word_df = spark.createDataFrame([
  (0, ['Hi', 'I', 'heard', 'about', 'Spark']),
  (1, ['I', 'wish', 'Java', 'could', 'use', 'case', 'classes']),
  (2, ['Logistic', 'regression', 'models', 'are', 'neat'])
], ['id', 'words'])

In [28]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')

In [29]:
ngram.transform(word_df).show()

+---+--------------------+--------------------+
 id| words| grams|
+---+--------------------+--------------------+
 0|[Hi, I, heard, ab...|[Hi I, I heard, h...|
 1|[I, wish, Java, c...|[I wish, wish Jav...|
 2|[Logistic, regres...|[Logistic regress...|
+---+--------------------+--------------------+

In [30]:
ngram.transform(word_df).select('grams').show(truncate=False)

+------------------------------------------------------------------+
grams |
+------------------------------------------------------------------+
[Hi I, I heard, heard about, about Spark] |
[I wish, wish Java, Java could, could use, use case, case classes]|
[Logistic regression, regression models, models are, are neat] |
+------------------------------------------------------------------+

#Part Two

In [32]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

###Creating a dataset

In [34]:
sen_df = spark.createDataFrame([
  (0.0, 'Hi I heard about Spark'),
  (0.0, 'I wish java could use case classes'),
  (1.0, 'Logistic regression models are neat') 
], ['label', 'sentence'])

In [35]:
sen_df.show()

+-----+--------------------+
label| sentence|
+-----+--------------------+
 0.0|Hi I heard about ...|
 0.0|I wish java could...|
 1.0|Logistic regressi...|
+-----+--------------------+

###Tokenizing

In [37]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [38]:
words_data = tokenizer.transform(sen_df)

In [39]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
label|sentence |words |
+-----+-----------------------------------+------------------------------------------+
0.0 |Hi I heard about Spark |[hi, i, heard, about, spark] |
0.0 |I wish java could use case classes |[i, wish, java, could, use, case, classes]|
1.0 |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+

###Getting the tf

In [41]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [42]:
featurized_data = hashing_tf.transform(words_data)

###Getting the idf

In [44]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [45]:
idf_model = idf.fit(featurized_data)

###Getting tf-idf

In [47]:
rescaled_data = idf_model.transform(featurized_data)

In [48]:
rescaled_data.select('label', 'features').show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
label|features |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
0.0 |(262144,[24417,49304,73197,91137,234657],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453]) |
0.0 |(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
1.0 |(262144,[13671,91006,132713,167122,190884],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453]) |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

###Creating a new sample df

In [50]:
df = spark.createDataFrame([
  (0, 'a b c'.split(' ')),
  (1, 'a b b c a'.split(' '))
], ['id', 'words'])

###Using CountVectorizer which counts the number of words using a vector

In [52]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

In [53]:
model = cv.fit(df)

In [54]:
result = model.transform(df)

In [55]:
result.show(truncate=False)

+---+---------------+-------------------------+
id |words |features |
+---+---------------+-------------------------+
0 |[a, b, c] |(3,[0,1,2],[1.0,1.0,1.0])|
1 |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

#Part 3

##Bulding a spam detection filter

###Initializing SparkSession

In [59]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

###Importing the dataset

In [61]:
df = spark.read.csv('/FileStore/tables/SMSSpamCollection', inferSchema=True, sep='\t')
df = df.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')
df.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

##Formatting the data

###Creating length column (feature engineering)

In [64]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer
from pyspark.sql.functions import length
df = df.withColumn('length', length(df['text']))
df.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

###Visualizing length col

In [66]:
df.groupBy('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

###Label encoding

In [68]:
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

###Using tokenizer (splitting the document list)

In [70]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')

###Removing irrelevant words

In [72]:
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')

###Counting the words

In [74]:
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')

###Getting TF-IDF

In [76]:
idf = IDF(inputCol='c_vec', outputCol='tf_idf')

###Getting transformed df

In [78]:
from pyspark.ml.feature import VectorAssembler
transformed_df = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')
transformed_df.show()

###Building a pipeline

In [80]:
from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,
                                  tokenizer,
                                  stop_remove,
                                  count_vec,
                                  idf,
                                  transformed_df])
cleaner = data_prep_pipe.fit(df)
clean_data = cleaner.transform(df)
clean_data = clean_data.select('label', 'features')
clean_data.show()

##Machine Learning

###Splitting the dataset

In [83]:
training, test = clean_data.randomSplit([0.7, 0.3])

###Building the model

In [85]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()
spam_detector = nb.fit(training)

In [86]:
df.printSchema()

root
-- class: string (nullable = true)
-- text: string (nullable = true)
-- length: integer (nullable = true)

###Making predictions

In [88]:
test_results = spam_detector.transform(test)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,2,7,8...|[-794.09202858104...|[1.0,1.5819402899...| 0.0|
 0.0|(13424,[0,1,2,41,...|[-1075.0666545329...|[1.0,1.9181323606...| 0.0|
 0.0|(13424,[0,1,9,14,...|[-538.65500227806...|[1.0,8.5862349265...| 0.0|
 0.0|(13424,[0,1,12,33...|[-449.52044523481...|[1.0,1.9906362722...| 0.0|
 0.0|(13424,[0,1,14,31...|[-220.50651348812...|[1.0,6.5811314241...| 0.0|
 0.0|(13424,[0,1,20,27...|[-982.73645814367...|[0.99999999880252...| 0.0|
 0.0|(13424,[0,1,24,31...|[-339.73632804117...|[1.0,1.0158872059...| 0.0|
 0.0|(13424,[0,1,43,69...|[-632.87798222039...|[2.42948026759760...| 1.0|
 0.0|(13424,[0,1,46,17...|[-1138.0070755193...|[6.21793336272282...| 1.0|
 0.0|(13424,[0,1,498,5...|[-322.96917413465...|[0.99999999992170...| 0.0|
 0.0|(13424,[0,1,874,1...|[-95.959483454264...|[0.99999997167794...| 0.0|
 0.0|(13424,[0,2,3,6,9...|[-3286.4438151157...|[1.0,3.4487478004...| 0.0|
 0.0|(13424,[0,2,4,5,7...|[-988.92568290548...|[1.0,3.5735192532...| 0.0|
 0.0|(13424,[0,2,4,7,2...|[-520.76457341955...|[1.0,2.2155758574...| 0.0|
 0.0|(13424,[0,2,4,8,1...|[-1331.9613370160...|[1.0,2.5909866949...| 0.0|
 0.0|(13424,[0,2,4,8,2...|[-1414.7353211374...|[1.0,1.7257045148...| 0.0|
 0.0|(13424,[0,2,4,25,...|[-436.69577132664...|[1.0,2.9841132609...| 0.0|
 0.0|(13424,[0,2,4,40,...|[-1586.6475764086...|[0.99938916402169...| 0.0|
 0.0|(13424,[0,2,4,44,...|[-1922.5955545566...|[1.0,7.4076288548...| 0.0|
 0.0|(13424,[0,2,5,8,4...|[-829.84047374545...|[1.0,2.2178490075...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

###Evaluting the model

In [90]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print('ACC of NB Model')
print(acc)

ACC of NB Model
0.9288385595770825

###Using evaluation based on my documentation

In [92]:
preds = spam_detector.transform(test)

In [93]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
area_under_curve = evaluator.evaluate(preds)
accuracy = MulticlassClassificationEvaluator(metricName='accuracy', labelCol='label')
accuracy = accuracy.evaluate(preds)

In [94]:
print(area_under_curve)
print(accuracy)

0.9352706299911269
0.9229411764705883